### 1. 모듈

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook
import time
import pandas as pd
import re

### 2. url 접속

In [2]:
driver = webdriver.Chrome('C:/Users/NK/Desktop/programming/python/chromedriver.exe')
url = 'http://likms.assembly.go.kr/bill/main.do'
driver.get(url)

In [3]:
search = driver.find_element_by_css_selector('li.gnbL2')
search.click()
time.sleep(2)

driver.find_element_by_css_selector('#AGE_AREA > select:nth-child(2) > option:nth-child(2)').click()

search = driver.find_element_by_css_selector('div.mt20.alignC > button.btnd')
search .click()
time.sleep(2)

more_bill = driver.find_element_by_css_selector('#pageSizeOption > option:nth-child(4)').click()

### 3. 크롤링

In [4]:
num_list = []
handled_list = []
title_list = []
proposer_type_list = []
propose_date_list = []
resolution_date_list = []
resolution_result_list = []
progress_list = []

proposer_list = []
paper_list = []
session_list = []
reason_list = []
board_list = []

In [5]:
n_times = 0

## 원하는 시작페이지
n_iter = 0

driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(11)').click()
while n_times <= n_iter - 2:
    driver.find_element_by_css_selector('#pageListViewArea > a:nth-child(13)').click()
    n_times +=1

In [6]:
start = 1
end = 11
## 10페이지가 끝나면 다음 10페이지를 불러옵니다.
while n_iter <= 300:
    ## start번째 페이지에 접속합니다.
    for k in tqdm_notebook(range(start,end)):
        driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({k})').click()
        ## start번째 페이지의 i번째 의안에 접근합니다.
        for i in range(1,101):
            ## i 번쨰 의안에 접근합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i})')

            num = bill.find_element_by_css_selector('td:nth-child(1)').text
            title = bill.find_element_by_css_selector('td.alignL').text
            proposer_type = bill.find_element_by_css_selector('td:nth-child(3)').text
            propose_date = bill.find_element_by_css_selector('td:nth-child(4)').text
            resolution_date = bill.find_element_by_css_selector('td:nth-child(5)').text
            resolution_result = bill.find_element_by_css_selector('td:nth-child(6)').text
            progress = bill.find_element_by_css_selector('td:nth-child(8)').text

            num_list.append(num)
            if resolution_date == '':
                handled_list.append('계')
            else:
                handled_list.append('처')
            title_list.append(title)
            proposer_type_list.append(proposer_type)
            propose_date_list.append(propose_date)
            resolution_date_list.append(resolution_date)
            resolution_result_list.append(resolution_result)
            progress_list.append(progress)


            ## start번째 페이지의 i번째 의안에 접속합니다.
            bill = driver.find_element_by_css_selector(f'body > div > div.contentWrap > div.subContents > div > div.tableCol01 > table > tbody > tr:nth-child({i}) > td.alignL > div.pl25 > a')
            bill.click()

            paper = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(4)').text
            session = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(5)').text     

            paper_list.append(paper)
            session_list.append(session)
            
            ## 제안이유를 담습니다.
            try:
                # 더 보기를 클릭합니다.
                more_info = driver.find_element_by_css_selector('#summaryContentMoreBtn').click()
                reason_raw_text = driver.find_element_by_css_selector('div.contIn > div.textType02').text
                
                content = []
                reason_split = re.split('\r|\t|\n|\xa0', reason_raw_text)
                content.append(''.join(reason_split))
                reason_list.append(content)            
                
            except:
                reason_list.append([]) 

            ## 소관위원회를 담습니다.
            try:
                board = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div:nth-child(5) > div > table > tbody > tr > td:nth-child(1)').text
                board_list.append(board)
            except:
                board_list.append([])
            
            ## 제안자를 담습니다.
            member_list = []    
            try:
                proposer = driver.find_element_by_css_selector('body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(3) > a > img')
                proposer.click()

                # 새로운 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[1])
                driver.get_window_position(driver.window_handles[1])

                member_num = len(driver.find_elements_by_tag_name('a'))
                member_end = (member_num//3) + member_num + 2

                for j in range(2, member_end):
                    try:
                        member = driver.find_element_by_css_selector(f'#periodDiv > div.layerInScroll.coaTxtScroll > div > a:nth-child({j})').text
                        member_list.append(member)
                    except:
                        continue
                        
                driver.close()
                proposer_list.append(member_list)

                # 원래 창으로 초점을 바꿉니다.
                driver.switch_to_window(driver.window_handles[0])
                driver.get_window_position(driver.window_handles[0])

            except:
                proposer_list.append([])

                
            driver.back()
            driver.execute_script("window.scrollTo(0, window.scrollY + 70)")
            
    driver.find_element_by_css_selector(f'#pageListViewArea > a:nth-child({10+start})').click()
    if n_iter == 0:
        start = 3; end = 13
    else :
        start = start; end = end
    n_iter += 1

C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:72: DeprecationWarning: use driver.switch_to.window instead
C:\Users\NK\Anaconda3\lib\site-packages\selenium\webdriver\remote\webdriver.py:1149: UserWarning: Only 'current' window is supported for W3C compatibile browsers.
  warnings.warn("Only 'current' window is supported for W3C compatibile browsers.")
C:\Users\NK\Anaconda3\lib\site-packages\ipykernel_launcher.py:89: DeprecationWarning: use driver.switch_to.window instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"body > div > div.contentWrap > div.subContents > div > div.contIn > div.tableCol01 > table > tbody > tr > td:nth-child(4)"}
  (Session info: chrome=84.0.4147.125)


### 4. 데이터프레임

In [11]:
result = [num_list, handled_list, title_list, proposer_type_list, propose_date_list, resolution_date_list, resolution_result_list,\
          progress_list, paper_list, session_list, reason_list, board_list, proposer_list]
result_df = pd.DataFrame(result).T
result_df.columns = ['의안번호', '처리여부', '의안명', '제안자구분', '제안일자', '의결일자', '의결결과', \
                     '심사진행상태', '문서', '제안회기' , '제안이유', '소관위원회', '제안자']
result_df

,의안번호,처리여부,의안명,제안자구분,제안일자,의결일자,의결결과,심사진행상태,문서,제안회기,제안이유,소관위원회,제안자
0,2102896,계,표시·광고의 공정화에 관한 법률 일부개정법률안(강훈식의원 등 10인),의원,2020-08-11,,,접수,,제21대 (2020~2024) 제380회,[],[],"[강훈식(더불어민주당/姜勳植), 고영인(더불어민주당/高永寅), 김승원(더불어민주당/..."
1,2102895,계,표시·광고의 공정화에 관한 법률 일부개정법률안(김두관의원 등 11인),의원,2020-08-11,,,접수,,제21대 (2020~2024) 제380회,[],[],"[김두관(더불어민주당/金斗官), 강훈식(더불어민주당/姜勳植), 고영인(더불어민주당/..."
2,2102894,계,국회법 일부개정법률안(조명희의원 등 12인),의원,2020-08-11,,,접수,,제21대 (2020~2024) 제380회,[제안이유 및 주요내용국회 운영과 관련하여 교섭단체 간 의견을 달리하거나 여야가 충...,[],"[조명희(미래통합당/曺明姬), 김기현(미래통합당/金起炫), 김석기(미래통합당/金碩基..."
3,2102893,계,아동복지법 일부개정법률안(인재근의원 등 13인),의원,2020-08-11,,,접수,,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행법에 따르면 시ㆍ도지사 및 시장ㆍ군수ㆍ구청장은 지역 중심의...,[],"[인재근(더불어민주당/印在謹), 강선우(더불어민주당/姜仙祐), 김영진(더불어민주당/..."
4,2102892,계,아동복지법 일부개정법률안(인재근의원 등 14인),의원,2020-08-11,,,접수,,제21대 (2020~2024) 제380회,"[제안이유 및 주요내용현행법에서는 아동복지시설의 장, 어린이집의 원장, 유치원의 원...",[],"[인재근(더불어민주당/印在謹), 강선우(더불어민주당/姜仙祐), 김병기(더불어민주당/..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,2102787,계,방송광고판매대행 등에 관한 법률 일부개정법률안,정부,2020-08-07,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용현행 피후견인 결격조항은 정신적 제약의 정도나 행위능력의 제한...,과학기술정보방송통신위원회,[]
110,2102786,계,지방세징수법 일부개정법률안(이성만의원 등 11인),의원,2020-08-07,,,접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용주택·상가 건물의 임대차 후 임대차건물이 경매나 공매로 처분되...,행정안전위원회,"[이성만(더불어민주당/李成萬), 기동민(더불어민주당/奇東旻), 김경만(더불어민주당/..."
111,2102785,계,국세징수법 일부개정법률안(이성만의원 등 11인),의원,2020-08-07,,,소관위접수,의안원문,제21대 (2020~2024) 제380회,[제안이유 및 주요내용주택·상가 건물의 임대차 후 임대차건물이 경매나 공매로 처분되...,기획재정위원회,"[이성만(더불어민주당/李成萬), 기동민(더불어민주당/奇東旻), 김경만(더불어민주당/..."
112,2102884,계,전기통신사업법 일부개정법률안(박성중의원 등 10인),의원,2020-08-11,,,접수,,제21대 (2020~2024) 제380회,[],[],"[박성중(미래통합당/朴成重), 김기현(미래통합당/金起炫), 김영식(미래통합당/金英植..."


In [ ]:
result3.to_csv('20대 국회 의안정보(5).csv', encoding='utf-8-sig', index=False)